# 05_Evaluate.ipynb : Évaluation des recommandations

Ce notebook a pour objectif de mesurer la qualité des listes de recommandations générées (CF, Content-Based, Hybrid) en les comparant aux interactions réelles du jeu de test.

**Étapes principales :**
1. Définition des métriques (Precision@K, NDCG@K).
2. Implémentation de fonctions utilitaires pour calculer ces métriques.
3. Fonction `evaluate` pour agréger les scores sur tous les utilisateurs.
4. Chargement des fichiers de recommandations et vérité terrain.
5. Calcul et affichage des résultats par modèle.

## 1. Imports des dépendances

In [1]:
import numpy as np
import pandas as pd

## 2. Fonctions d'évaluation
### 2.1. Precision@K

Calcule le ratio d'items dans les k premières positions qui apparaissent dans la vérité terrain.

In [2]:
def precision_at_k(ranked_list, ground_truth, k):
    """
    :param ranked_list: liste des item_id recommandés classés
    :param ground_truth: ensemble des item_id réellement vus
    :param k: nombre de positions à considérer
    :return: précision
    """
    hit = sum(1 for item in ranked_list[:k] if item in ground_truth)
    return hit / k

### 2.2. DCG@K et NDCG@K

- **DCG@K** : somme des gains pondérés par le log de la position.
- **IDCG@K** : DCG maximum possible (scénario idéal).
- **NDCG@K** : DCG normalisé par IDCG.

In [3]:
def dcg_at_k(ranked_list, ground_truth, k):
    dcg = 0.0
    for i, item in enumerate(ranked_list[:k]):
        if item in ground_truth:
            dcg += 1.0 / np.log2(i + 2)
    return dcg


def idcg_at_k(ground_truth, k):
    ideal_hits = min(len(ground_truth), k)
    return sum(1.0 / np.log2(i + 2) for i in range(ideal_hits))


def ndcg_at_k(ranked_list, ground_truth, k):
    idcg = idcg_at_k(ground_truth, k)
    if idcg == 0:
        return 0.0
    return dcg_at_k(ranked_list, ground_truth, k) / idcg

## 3. Fonction d'évaluation globale

La fonction `evaluate` parcourt chaque utilisateur, récupère ses recommandations et la vérité terrain, puis calcule la précision et le NDCG, et renvoie la moyenne globale.

In [4]:
def evaluate(recs_df, truth_df, k=10):
    """
    :param recs_df: DataFrame avec colonnes ['user_id','item_id','rank'] des recommandations
    :param truth_df: DataFrame avec colonnes ['user_id','item_id'] des interactions réelles
    :param k: cutoff pour les métriques
    :return: dict avec les scores moyens {'precision': ..., 'ndcg': ...}
    """
    users = recs_df['user_id'].unique()
    precisions, ndcgs = [], []
    for u in users:
        preds = recs_df[recs_df.user_id == u].sort_values('rank')['item_id'].tolist()
        actual = set(truth_df[truth_df.user_id == u]['item_id'])
        precisions.append(precision_at_k(preds, actual, k))
        ndcgs.append(ndcg_at_k(preds, actual, k))
    return {
        'precision': np.mean(precisions),
        'ndcg': np.mean(ndcgs)
    }

## 4. Chargement des données et évaluation des modèles

On charge la vérité terrain (`interactions_test.csv` ou `small_matrix.csv`) et les fichiers de recommandations (`submission_cf.csv`, etc.), puis on affiche les scores pour chaque approche.

In [5]:
# Chargement de la vérité terrain
truth = pd.read_csv("../data/small_matrix.csv").rename(columns={"video_id": "item_id"})

# Liste des modèles à évaluer
models = ["cf", "content", "hybrid"]
for name in models:
    # Chargement des recommandations
    recs = pd.read_csv(f"submission_{name}.csv").rename(columns={"video_id": "item_id"})
    # Calcul des métriques
    scores = evaluate(recs, truth, k=10)
    # Affichage des résultats
    print(f"📊 Modèle {name.upper()}")
    print(f"  - Precision@10 : {scores['precision']:.4f}")
    print(f"  - NDCG@10      : {scores['ndcg']:.4f}\n")

📊 Modèle CF
  - Precision@10 : 0.4505
  - NDCG@10      : 0.4884

📊 Modèle CONTENT
  - Precision@10 : 0.4851
  - NDCG@10      : 0.5547

📊 Modèle HYBRID
  - Precision@10 : 0.4825
  - NDCG@10      : 0.5262

